<a href="https://colab.research.google.com/github/lipeng2021/-python1/blob/main/6_1%E8%AF%AD%E8%A8%80%E6%A8%A1%E5%9E%8B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import torch
import torch.nn as nn
import random
import zipfile

In [10]:
with zipfile.ZipFile('/home/jaychou_lyrics.txt.zip')as zin:
   with zin.open('jaychou_lyrics.txt')as f:
     corpus_chars = f.read().decode('utf-8')
corpus_chars[:40]

'想要有直升机\n想要和你飞到宇宙去\n想要和你融化在一起\n融化在宇宙里\n我每天每天每'

In [11]:
corpus_chars= corpus_chars.replace('\n',' ').replace('\r',' ')
corpus_chars = corpus_chars[:10000]



In [12]:
idx_to_char = list(set(corpus_chars))
char_to_idx = dict([(char,i)for i,char in enumerate(idx_to_char)])
vocab_size = len(char_to_idx)
print(vocab_size)
corpus_indices = [char_to_idx[char]for char in corpus_chars]
sample = corpus_indices[:20]
print('char:',corpus_chars[:20])
print('indices:',sample)

1027
char: 想要有直升机 想要和你飞到宇宙去 想要和
indices: [538, 25, 996, 95, 779, 998, 620, 538, 25, 296, 685, 457, 505, 455, 585, 847, 620, 538, 25, 296]


In [13]:
def load_data_jay_lyrics():
  with zipfile.ZipFile('/home/jaychou_lyrics.txt.zip') as zin:
    with zin.open('jaychou_lyrics.txt')as f:
      corpus_chars = f.read().decode('utf-8')
  corpus_chars = corpus_chars.replace('\n',' ').replace('\r',' ')
  corpus_chars = corpus_chars[:10000]
  idx_to_char = list(set(corpus_chars))
  char_to_idx = dict([(char,i)for i,char in enumerate(idx_to_char)])
  vocab_size = len(char_to_idx)
  corpus_indics = [char_to_idx[char]for char in corpus_chars]
  return corpus_indices,char_to_idx,idx_to_char,vocab_size

随机采样：

In [35]:
def data_iter_random(corpus_indices,batch_size,num_steps,device=None):
  num_examples = (len(corpus_indices)-1)//num_steps
  print('num_examples=',num_examples)
  epoch_size = num_examples//batch_size
  print('epoch_size=',epoch_size)
  example_indices = list(range(num_examples))
  random.shuffle(example_indices)
  print('example_indices=',example_indices)
  def _data(pos):
    return corpus_indices[pos:pos+num_steps]
  if device is None:
    device = torch.device('cuda'if torch.cuda.is_available()else 'cpu')
    print('device=',device)
  for i in range(epoch_size):
    i = i*batch_size
    batch_indices = example_indices[i:i+batch_size]
    print('batch_indices=',batch_indices)
    X = [_data(j*num_steps)for j in batch_indices]
    print('X=',X)
    Y = [_data(j*num_steps+1)for j in batch_indices]
    yield torch.tensor(X,dtype=torch.float32,device=device)
    yield torch.tensor(Y,dtype=torch.float32,device=device)
  

In [37]:
my_seq=list(range(30))
num = 0 
for X,Y in data_iter_random(my_seq,batch_size=2,num_steps=6):
  print(X)
  print(Y)

num_examples= 4
epoch_size= 2
example_indices= [3, 0, 2, 1]
device= cpu
batch_indices= [3, 0]
X= [[18, 19, 20, 21, 22, 23], [0, 1, 2, 3, 4, 5]]
tensor([18., 19., 20., 21., 22., 23.])
tensor([0., 1., 2., 3., 4., 5.])
tensor([19., 20., 21., 22., 23., 24.])
tensor([1., 2., 3., 4., 5., 6.])
batch_indices= [2, 1]
X= [[12, 13, 14, 15, 16, 17], [6, 7, 8, 9, 10, 11]]
tensor([12., 13., 14., 15., 16., 17.])
tensor([ 6.,  7.,  8.,  9., 10., 11.])
tensor([13., 14., 15., 16., 17., 18.])
tensor([ 7.,  8.,  9., 10., 11., 12.])


相邻采样：

In [46]:
from torch.cuda import is_available
def data_iter_consecutive(corpus_indices,batch_size,num_steps,device=None):
  if device==None:
    device = torch.device('cuda'if (torch.cuda.is_available())else 'cpu')
  corpus_indices = torch.tensor(corpus_indices,dtype=torch.float32,device=device)
  data_len = len(corpus_indices)
  print('data_len:',data_len)
  batch_len = data_len//batch_size
  print('batch_len:',batch_len)
  indices =corpus_indices[0:batch_size*batch_len].view(batch_size,batch_len)
  print('indices',indices)
  epoch_size = (batch_len-1)//num_steps
  print('epoch_size',epoch_size)
  for i in range(epoch_size):
    i = i*num_steps
    X = indices[:,i:i+num_steps]
    Y = indices[:,i+1:i+num_steps+1]
    yield X,Y

In [ ]:
my_seq=list(range(30))
num = 0
for X,Y in data_iter_consecutive(my_seq,batch_size=2,num_steps=6):
  print(X)
  print(Y)